In [23]:
import sys; sys.path.append('../../')
import numpy as np
import pandas as pd
from DataPreparation.DataPreparation import read_data
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from sklearn.model_selection import cross_val_predict
from utils import load_hyperparameters, save_model, get_metrics
from ModelAnalysis import cross_validation
from sklearn.ensemble import RandomForestClassifier
model_name= 'RandomForest'

mlq.start_quest("RandomForest", table_dest="../../", log_defs=False)

# <font color="aqua">Random Forest</font> Model

### Read the data

In [24]:
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical')
opt_params = load_hyperparameters(model_name)

### Initiate model

In [25]:
clf = mlq.l(RandomForestClassifier)(**opt_params)

### Evaluate Model Bias

In [26]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

              precision    recall  f1-score   support

           0      0.987     0.974     0.980       152
           1      0.963     0.969     0.966       160
           2      0.988     0.994     0.991       324
           3      1.000     0.998     0.999       544

    accuracy                          0.990      1180
   macro avg      0.984     0.984     0.984      1180
weighted avg      0.990     0.990     0.990      1180



### Evaluate Model Generalization

In [27]:
loo, kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1)

loo_acc=loo.get('loo_acc')
loo_wf1=loo.get('loo_wf1')
repeated_10fold_acc, repeated_10fold_wf1 = kfold.get(f'10-Repeated 10-fold')[:2]

loo_acc,10-Repeated 10-fold
0.9559,0.9534


loo_wf1,10-Repeated 10-fold
0.9554,0.9528


#### Save Model

In [28]:
save_model(model_name, clf)

### Tracking

In [29]:
mlq.log_metrics(train_acc, train_wf1,loo_acc, loo_wf1, repeated_10fold_acc, repeated_10fold_wf1)
mlq.end_quest()
mlq.show_logs("RandomForest", table_dest="../../", last_k=6)